In [1]:
import pandas as pd
import os

In [3]:
# path setting
data_folder = r"C:\Users\saru8\OneDrive\Documents\DeliverIQ\data"
csv_file = os.path.join(data_folder, "supply_chain_merged.csv")

In [5]:
# output folder for cleaned dataset
output_folder = r"C:\Users\saru8\OneDrive\Documents\DeliverIQ\outputs"
os.makedirs(output_folder, exist_ok=True)
output_csv = os.path.join(output_folder, "supply_chain_prepared.csv")

In [6]:
# Loading CSV
df = pd.read_csv(csv_file)
print("Dataset loaded successfully!\n")

Dataset loaded successfully!



In [7]:
# Preview dataset
print("First 5 rows:")
print(df.head())
print("\nDataset info:")
print(df.info())

First 5 rows:
    Order_ID  Order_Date   Ship_Date Planned_Delivery_Date  \
0  ORD100000  13-04-2025  13-04-2025            19-04-2025   
1  ORD100001  17-04-2025  18-04-2025            25-04-2025   
2  ORD100002  13-03-2025  13-03-2025            20-03-2025   
3  ORD100003  08-07-2025  08-07-2025            12-07-2025   
4  ORD100004  21-01-2025  22-01-2025            29-01-2025   

  Actual_Delivery_Date  Delay_Days     Product     Supplier     Region  \
0           19-04-2025           0  Product_44  Supplier_26  Hyderabad   
1           28-04-2025           3  Product_46  Supplier_13      North   
2           22-03-2025           2  Product_97  Supplier_13       West   
3           14-07-2025           2  Product_98   Supplier_2  Bengaluru   
4           31-01-2025           2  Product_68  Supplier_27  Hyderabad   

  Warehouse  ...  Revenue  Transport_Mode     Courier Damage_Report  \
0     WH_10  ...  1177.51            Road  Courier_78             0   
1      WH_9  ...  2134.11 

In [8]:
# Convert date columns
date_cols = ['Order_Date', 'Ship_Date', 'Planned_Delivery_Date', 'Actual_Delivery_Date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

C:\Users\saru8\AppData\Local\Temp\ipykernel_19652\2858708526.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\saru8\AppData\Local\Temp\ipykernel_19652\2858708526.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\saru8\AppData\Local\Temp\ipykernel_19652\2858708526.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\saru8\AppData\Local\Temp\ipykernel_19652\2858708526.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specif

In [9]:
date_cols = ['Order_Date', 'Ship_Date', 'Planned_Delivery_Date', 'Actual_Delivery_Date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')

In [10]:
# Delay in days
df['Delay_Days'] = (df['Actual_Delivery_Date'] - df['Planned_Delivery_Date']).dt.days

In [11]:
# On-Time delivery flag
df['On_Time'] = df['Delay_Days'].apply(lambda x: 1 if x <= 0 else 0)

In [12]:
# Lead time in days
df['Lead_Time_days'] = (df['Actual_Delivery_Date'] - df['Order_Date']).dt.days

In [13]:
# Cost per unit
df['Cost_per_unit'] = df['Cost'] / df['Quantity']

In [14]:
# Calculate KPIs
print("\n--- Key KPIs ---")
# On-Time %
on_time_pct = df['On_Time'].mean() * 100
print(f"On-Time Delivery %: {on_time_pct:.2f}%")


--- Key KPIs ---
On-Time Delivery %: 19.91%


In [15]:
# Average Lead Time
avg_lead_time = df['Lead_Time_days'].mean()
print(f"Average Lead Time (days): {avg_lead_time:.2f}")

Average Lead Time (days): 8.92


In [16]:
# Average Delay Days (only delayed orders)
avg_delay_days = df.loc[df['Delay_Days'] > 0, 'Delay_Days'].mean()
print(f"Average Delay Days: {avg_delay_days:.2f}")

Average Delay Days: 2.25


In [17]:
# Average Cost per Delivery
avg_cost_per_delivery = df['Cost'].mean()
print(f"Average Cost per Delivery: {avg_cost_per_delivery:.2f}")

Average Cost per Delivery: 1384.92


In [18]:
# Supplier Reliability Index
supplier_reliability = df.groupby('Supplier')['On_Time'].mean() * 100
print("\nSupplier Reliability Index (% on-time):")
print(supplier_reliability.sort_values(ascending=False))


Supplier Reliability Index (% on-time):
Supplier
Supplier_14    34.883721
Supplier_16    32.000000
Supplier_22    30.852995
Supplier_12    30.409357
Supplier_27    29.806714
Supplier_26    29.514563
Supplier_28    28.985507
Supplier_15    28.959276
Supplier_19    28.495102
Supplier_21    28.351648
Supplier_29    27.903469
Supplier_6     27.639155
Supplier_8     27.385892
Supplier_11    27.321669
Supplier_2     26.937046
Supplier_18    26.891253
Supplier_30    26.229508
Supplier_4     25.910364
Supplier_1     25.870647
Supplier_23    25.401460
Supplier_9      5.128205
Supplier_7      3.928571
Supplier_20     3.805774
Supplier_3      3.203661
Supplier_24     3.035144
Supplier_10     2.923977
Supplier_5      2.752294
Supplier_13     0.000000
Supplier_17     0.000000
Supplier_25     0.000000
Name: On_Time, dtype: float64


In [19]:
# Saving cleaned dataset
df.to_csv(output_csv, index=False)
print(f"\nPrepared dataset saved at: {output_csv}")


Prepared dataset saved at: C:\Users\saru8\OneDrive\Documents\DeliverIQ\outputs\supply_chain_prepared.csv


In [20]:
# Check that these columns exist:
import pandas as pd

output_csv = r"C:\Users\saru8\OneDrive\Documents\DeliverIQ\outputs\supply_chain_prepared.csv"
df = pd.read_csv(output_csv)

print(df.head())
print(df.columns)

    Order_ID  Order_Date   Ship_Date Planned_Delivery_Date  \
0  ORD100000  2025-04-13  2025-04-13            2025-04-19   
1  ORD100001  2025-04-17  2025-04-18            2025-04-25   
2  ORD100002  2025-03-13  2025-03-13            2025-03-20   
3  ORD100003  2025-07-08  2025-07-08            2025-07-12   
4  ORD100004  2025-01-21  2025-01-22            2025-01-29   

  Actual_Delivery_Date  Delay_Days     Product     Supplier     Region  \
0           2025-04-19           0  Product_44  Supplier_26  Hyderabad   
1           2025-04-28           3  Product_46  Supplier_13      North   
2           2025-03-22           2  Product_97  Supplier_13       West   
3           2025-07-14           2  Product_98   Supplier_2  Bengaluru   
4           2025-01-31           2  Product_68  Supplier_27  Hyderabad   

  Warehouse  ...  Revenue  Transport_Mode     Courier Damage_Report  \
0     WH_10  ...  1177.51            Road  Courier_78             0   
1      WH_9  ...  2134.11            Roa